# 혼용률 인코더 학습

## 라이브러리 로드

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split

import  os, json
from tqdm import tqdm 

from dataset import dataset
from models.encoder.simple_ae import SimpleAE
from utils.model_controler import run_train_test_1_input
from utils.custom_logger import get_logger

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

## 데이터 로드

In [ ]:
portion_dataset = torch.load('dataset/portion_dataset.pth')

In [ ]:
train_size = int(len(portion_dataset)*0.8)
test_size = len(portion_dataset)-train_size

train, test = random_split(portion_dataset, [train_size, test_size])

In [ ]:
train_loader = DataLoader(train, batch_size=256, shuffle=True)
test_loader = DataLoader(test, batch_size=256, shuffle=False)

# 모델 로드

In [ ]:
model = SimpleAE(input_dim=4, latent_dim=512)

for i in model.parameters():
  i.requires_grad = True

model.train()
model.to(device)

## 하이퍼 파라미터 세팅

In [ ]:
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)
num_epochs = 100
test_step = 10
logger = get_logger('autoencoder_portion')

## 모델 학습

In [ ]:
run_train_test_1_input(
    model=model, 
    train_loader=train_loader, 
    test_loader=test_loader, 
    criterion=criterion, 
    optimizer=optimizer, 
    num_epochs=num_epochs, 
    test_step=test_step, 
    logger=logger, 
    device=device,
    path='.checkpoints/best/ae_portion_best.pth'
)